In [17]:
import requests 
import os  
import pandas as pd 

os.environ["DASK_EXPERIMENTAL__PYARROW_STRINGS"] = "false"



In [3]:
taxonomy = pd.read_csv("data/nucc_taxonomy_250.csv")

taxonomy.head()

taxonomy.info()




<class 'pandas.core.frame.DataFrame'>
RangeIndex: 879 entries, 0 to 878
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Code            879 non-null    object
 1   Grouping        879 non-null    object
 2   Classification  879 non-null    object
 3   Specialization  639 non-null    object
 4   Definition      858 non-null    object
 5   Notes           597 non-null    object
 6   Display Name    879 non-null    object
 7   Section         879 non-null    object
dtypes: object(8)
memory usage: 55.1+ KB


In [7]:
taxonomy["Classification"].unique()


array(['Multi-Specialty', 'Single Specialty', 'Allergy & Immunology',
       'Anesthesiology', 'Clinical Pharmacology',
       'Colon & Rectal Surgery', 'Dermatology',
       'Electrodiagnostic Medicine', 'Emergency Medicine',
       'Family Medicine', 'General Practice', 'Hospitalist',
       'Independent Medical Examiner', 'Integrative Medicine',
       'Internal Medicine', 'Legal Medicine', 'Medical Genetics',
       'Neurological Surgery', 'Neuromusculoskeletal Medicine & OMM',
       'Neuromusculoskeletal Medicine, Sports Medicine',
       'Nuclear Medicine', 'Obstetrics & Gynecology', 'Ophthalmology',
       'Oral & Maxillofacial Surgery', 'Orthopaedic Surgery',
       'Otolaryngology', 'Pain Medicine', 'Pathology', 'Pediatrics',
       'Phlebology', 'Physical Medicine & Rehabilitation',
       'Plastic Surgery', 'Preventive Medicine', 'Psychiatry & Neurology',
       'Radiology', 'Surgery',
       'Thoracic Surgery (Cardiothoracic Vascular Surgery)',
       'Transplant Surgery',

In [8]:
def filter_taxonomy_by_codes(taxonomy_df):
    codes = [
        "320800000X",  # Community Based Residential Treatment Facility, Mental Illness
        "320900000X",  # Community Based Residential Treatment Facility, Mental Retardation and/or Developmental Disabilities
        "323P00000X",  # Psychiatric Residential Treatment Facility
        "322D00000X",  # Residential Treatment Facility, Emotionally Disturbed Children
        "320600000X",  # Residential Treatment Facility, Mental Retardation and/or Developmental Disabilities
        "320700000X",  # Residential Treatment Facility, Physical Disabilities
        "324500000X",  # Substance Abuse Rehabilitation Facility
        "3245S0500X",  # Substance Abuse Rehabilitation Facility — Substance Abuse Treatment, Children
    ]
    return taxonomy_df[taxonomy_df["Code"].isin(codes)] 



only_substance_abuse = filter_taxonomy_by_codes(taxonomy) 

only_substance_abuse.head()


,Code,Grouping,Classification,Specialization,Definition,Notes,Display Name,Section
818,320800000X,Residential Treatment Facilities,Community Based Residential Treatment Facility...,NaN,A home-like residential facility providing psy...,[7/1/2003: new],Mental Illness Community Based Residential Tre...,Non-Individual
819,320900000X,Residential Treatment Facilities,Community Based Residential Treatment Facility...,NaN,A home-like residential facility providing hab...,"[7/1/2003: new, 1/1/2021: modified title, modi...",Intellectual and/or Developmental Disabilities...,Non-Individual
820,323P00000X,Residential Treatment Facilities,Psychiatric Residential Treatment Facility,NaN,A residential treatment facility (RTF) is a fa...,"Source: Champus Policy manual, Volume II, p. 6...",Psychiatric Residential Treatment Facility,Non-Individual
821,322D00000X,Residential Treatment Facilities,"Residential Treatment Facility, Emotionally Di...",NaN,A residential facility that provides habilitat...,Source: U.S. Department of Defense Regulation ...,Emotionally Disturbed Childrens' Residential T...,Non-Individual
822,320600000X,Residential Treatment Facilities,"Residential Treatment Facility, Intellectual a...",NaN,A residential facility that provides habilitat...,[7/1/2003: new. 1/1/2021: modified title and d...,Intellectual and/or Developmental Disabilities...,Non-Individual


In [9]:
os.listdir(r"C:\Users\Pinda\Downloads\NPPES_Data_Dissemination_May_2025_V2")

['endpoint_pfile_20050523-20250511.csv',
 'endpoint_pfile_20050523-20250511_fileheader.csv',
 'npidata_pfile_20050523-20250511.csv',
 'npidata_pfile_20050523-20250511_fileheader.csv',
 'NPPES_Data_Dissemination_CodeValues.pdf',
 'NPPES_Data_Dissemination_Readme_v.2.pdf',
 'othername_pfile_20050523-20250511.csv',
 'othername_pfile_20050523-20250511_fileheader.csv',
 'pl_pfile_20050523-20250511.csv',
 'pl_pfile_20050523-20250511_fileheader.csv']

In [18]:
import dask.dataframe as dd
from dask.diagnostics import ProgressBar 



In [20]:
ProgressBar().register()

# Target taxonomy codes
TARGET_CODES = {
    "320800000X", "320900000X", "323P00000X", "322D00000X",
    "320600000X", "320700000X", "324500000X", "3245S0500X",
}


# 📂 Path to the large npidata file (V.2)
csv_path = r"C:\Users\Pinda\Downloads\NPPES_Data_Dissemination_May_2025_V2\npidata_pfile_20050523-20250511.csv"

# 🔧 Use Dask to load with type coercion
ddf = dd.read_csv(csv_path, dtype=str, blocksize="64MB", assume_missing=True)

# Filter to organizations only
ddf = ddf[ddf["Entity Type Code"] == "2"]

# Grab all taxonomy code columns
taxonomy_cols = [col for col in ddf.columns if "Healthcare Provider Taxonomy Code_" in col]

# Define filter function to apply across rows (parallelized)
def has_matching_taxonomy(row):
    return any(code in TARGET_CODES for code in row if isinstance(code, str))

# Apply filtering in parallel (meta=bool required)
filtered_ddf = ddf[ddf[taxonomy_cols].apply(has_matching_taxonomy, axis=1, meta=bool)]

# Write partitioned output (fastest)
filtered_ddf.to_csv("rehab_partitions/filtered-*.csv", index=False)

ImportError: An error occurred while calling the read_csv method registered to the pandas backend.
Original Message: pyarrow>=10.0.1 is required for PyArrow backed StringArray.